In [2]:
# Get tone ukrainian dict
# https://github.com/lang-uk/tone-dict-uk

import pandas as pd

tone_dict_df = pd.read_csv('tone-dict-uk.tsv', sep='\t')
tone_dict_df = tone_dict_df.astype({"tone_value": float})

TONE_DICT_UK = tone_dict_df.set_index('word').to_dict()['tone_value']

In [3]:
!pip install fasttext-langdetect

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Detect language

from ftlangdetect import detect
import string

def sanitize_text(text):
    text = text.replace(chr(769), '')
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Returns 'uk', 'en', 'ru'
def get_lang(line):
    sanitized_line = sanitize_text(line) 
    return detect(sanitized_line, low_memory=True)['lang']

In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

SIA_UK = SentimentIntensityAnalyzer()
SIA_UK.lexicon.update(TONE_DICT_UK)

SIA_EN = SentimentIntensityAnalyzer()

def get_polarity_scores(text):
    lang = get_lang(text)
    if lang == 'en':
        return SIA_EN.polarity_scores(text)
    return SIA_UK.polarity_scores(text)

def classify_polarity_scores(polarity_scores):
    compound = polarity_scores['compound']
    result = "NEUTRAL"
    if compound > 0.05:
        result = "POSITIVE"
    else:
        result = "NEGATIVE"
    return result

[nltk_data] Downloading package stopwords to /home/mlgmag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mlgmag/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mlgmag/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
!pip install git+https://github.com/kmike/pymorphy2.git
!pip install -U pymorphy2-dicts-uk

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/kmike/pymorphy2.git to /tmp/pip-req-build-j16mmkz3
  Running command git clone --filter=blob:none --quiet https://github.com/kmike/pymorphy2.git /tmp/pip-req-build-j16mmkz3
  Resolved https://github.com/kmike/pymorphy2.git to commit 92d546f042ff14601376d3646242908d5ab786c1
  Preparing metadata (setup.py) ... done
Defaulting to user installation because normal site-packages is not writeable


In [16]:
import pymorphy2
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
ENGLISH_STOP_WORDS = stopwords.words('english')

morph = pymorphy2.MorphAnalyzer(lang='uk')

def to_raw_uk(text):
    sentences = nltk.sent_tokenize(text)

    normal_words = []

    for sentence in sentences:
        sanitized_sentence = sanitize_text(sentence)
        tokens = nltk.word_tokenize(sanitized_sentence)
        for token in tokens:        
            parse_result = morph.parse(token)[0]
            normal_form = parse_result.normal_form
            normal_words.append(normal_form)

    return " ".join(normal_words)



ps = PorterStemmer()

def to_raw_en(text):
    sentences = nltk.sent_tokenize(text)

    normal_words = []

    for sentence in sentences:
        sanitized_sentence = sanitize_text(sentence)
        tokens = nltk.word_tokenize(sanitized_sentence)
        filtered_tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]
        for token in filtered_tokens:
            normal_form = ps.stem(token)
            normal_words.append(normal_form)

    return " ".join(normal_words)


[nltk_data] Downloading package stopwords to /home/mlgmag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
def demo(text):
    lang = get_lang(text)
    raw_text = ""
    if lang == 'en':
        raw_text = to_raw_en(text)
    elif lang == 'uk':
        raw_text = to_raw_ua(text)

    polarity_scores = get_polarity_scores(raw_text)
    polarity = classify_polarity_scores(polarity_scores)

    print(f"Initial text:\t\t '{text}'")
    print(f"Raw text:\t\t '{raw_text}'")
    print(f"Polarity:\t\t '{polarity}'")
    print(f"Polarity details:\t '{polarity_scores}'")

TEXT1 = "У Зеленського вважають, що вандали під ОП хотіли спровокувати насильство"
TEXT2 = "Книга Елізабет Гілберт \"Їсти. Молитися. Кохати\" після її прочитання викликала відчуття легкості та одночасно потребу в зануренні та вивченні власного \"Я\". Це є вважаю однозначним плюсом. Мимоволі, в процесі читання, відчувається бажання вдосконалити себе, знайти те, що зможе відкриє тебе, а ще те, що принесе нову та потрібну радість кожного дня у вже буденне життя.\nОсобливістю даного роману є його автобіографічність, що завжди приваблює. Дійсно цікаво спостерігати за тим, що переживала людина у важливий для себе життєвий період.\nВарто відмітити зручність структури роману, адже вся книга поділена на три частини, відповідно з трьома країнами, що їх відвідала авторка. Кожна з цих частин ділиться на маленькі розділи, що є дуже зручним у процесі читання. Загалом, і оформлення книги, і друк тексту, на приємних для очей молочного кольору аркушах, однозначно приваблюють.\nСама історія дуже підходить для літніх читань на природі і, звісно, більше для жінок, ніж для чоловіків. Хоча, хто зна. Може чоловіки саме тут зможуть відкрити для себе загадковий жіночий світ."
TEXT3 = "This is a great gpu with very impressive cooling. Mine runs at 60 degrees Celsius under full load and maxes out at 215 watts. It performs identically to my 3080 at a fraction of the power consumption. The provided gpu anti sag mount is a nice plus and one of the better things in my opinion is the plug is inset a bit to help the angle not be so hard on the 16 pin 12vhp cable. The manual also specifies to use 2 separate 8pin connectors rather than one and a daisy chain."
TEXT4 = "Очень рада, что столь известный роман Элизабет Гилберт можно теперь прочитать и на украинском. Переводом осталась довольна, хоть и есть несколько спорных моментов (изначально читала его в оригинале и было с чем сравнить). Немного странным кажется выбор обложки — как правило, его публикуют в чем-то ярком и пестром, но тут, должна признать, неплохо передается внутренние переживания героини в самом начале.\n\nСразу хочу предупредить, что если Вы не можете себе представить что можно бросить всё — дом, работу, мужа, семью — и отправиться путешествовать просто потому что не можете вновь почувствовать вкус счастья, то лучше не читать эту книгу. Она вызовет только раздражение, а проблемы главной героини покажутся не такими уж серьезными. Но этот роман может оказаться настоящим бальзамом для души тех женщин, которые сбились с пути или зашли в тупик жизни.\n\nЛиз — уже немолода. У неё есть хорошая работа и любящий муж. Есть и любовник. Но ничто уже счастья ей не приносит. Дабы справится с депрессией, она решает отправиться в путешествие и посетить Италию, Индию и Бали. Гилберт приглашает в это путешествие и нас.\n\nИталия — страна солнца и вкуса, родина пиццы, пасты и других аппетитных вещей. Здесь главная героиня познает наконец вкус жизни, учиться наслаждаться едой и принимать себя даже не влезая в джинсы)) Да, еде в этой части отведено много места. Лично я, пока читала, сама пристрастилась к итальянской кухне.\n\nВ Индии Лиз пытается найти душевное равновесие посредством молитв и медитаций. Тут мы узнаем немало об индийской культуре и её жителях, а также тех, кто как и Лиз, пытается найти себя.\n\nБали... Бали создано для любви) И там героиня Гилберт смогла найти того самого, в которого можно по уши влюбиться как в юности, со всей страстью.\n\nКнига полна красок и учит жить. Возможно, не пускаясь восвояси по разным странам и континентам, но помогает найти ключи к счастью вокруг.\n\nРекомендую как книгу для отпуска, так и книгу от депрессии. А также просто как добротный женский роман для поклонников этого жанра!)"

demo(TEXT2)

Initial text:		 'Книга Елізабет Гілберт "Їсти. Молитися. Кохати" після її прочитання викликала відчуття легкості та одночасно потребу в зануренні та вивченні власного "Я". Це є вважаю однозначним плюсом. Мимоволі, в процесі читання, відчувається бажання вдосконалити себе, знайти те, що зможе відкриє тебе, а ще те, що принесе нову та потрібну радість кожного дня у вже буденне життя.
Особливістю даного роману є його автобіографічність, що завжди приваблює. Дійсно цікаво спостерігати за тим, що переживала людина у важливий для себе життєвий період.
Варто відмітити зручність структури роману, адже вся книга поділена на три частини, відповідно з трьома країнами, що їх відвідала авторка. Кожна з цих частин ділиться на маленькі розділи, що є дуже зручним у процесі читання. Загалом, і оформлення книги, і друк тексту, на приємних для очей молочного кольору аркушах, однозначно приваблюють.
Сама історія дуже підходить для літніх читань на природі і, звісно, більше для жінок, ніж для чоловіків. Хо